# Whitelee wind farm



In [1]:
import requests
import json
from types import SimpleNamespace

import pickle
import os
import re

import openstreetmap_mapping as osm

import pandas as pd
import numpy as np
import matplotlib as mpl

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import WMTSTileSource
from bokeh.models import HoverTool
from bokeh.models import LabelSet
from bokeh.palettes import viridis, Category20, Blues256
from bokeh.palettes import Category10

output_notebook()

from pyproj import Transformer

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")

In [4]:
def luminance(rgb):
    """Calculates the brightness of an rgb 255 color. See https://en.wikipedia.org/wiki/Relative_luminance

    Args:
        rgb(:obj:`tuple`): 255 (red, green, blue) tuple

    Returns:
        luminance(:obj:`scalar`): relative luminance

    Example:

        .. code-block:: python

            >>> rgb = (255,127,0)
            >>> luminance(rgb)
            0.5687976470588235

            >>> luminance((0,50,255))
            0.21243529411764706

    """

    luminance = (0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]) / 255

    return luminance

In [5]:
def color_to_rgb(color):
    """Converts named colors, hex and normalised RGB to 255 RGB values

    Args:
        color(:obj:`color`): RGB, HEX or named color

    Returns:
        rgb(:obj:`tuple`): 255 RGB values

    Example:

        .. code-block:: python

            >>> color_to_rgb("Red")
            (255, 0, 0)

            >>> color_to_rgb((1,1,0))
            (255,255,0)

            >>> color_to_rgb("#ff00ff")
            (255,0,255)
    """

    if isinstance(color, tuple):
        if max(color) > 1:
            color = tuple([i / 255 for i in color])

    rgb = mpl.colors.to_rgb(color)

    rgb = tuple([int(i * 255) for i in rgb])

    return rgb


In [6]:
def plot_points(df_points,
    tile_name="OpenMap",
    plot_width=800,
    plot_height=800,
    marker_size=14,
    kwargs_for_figure={},
    kwargs_for_marker={},
):
    """Plot the windfarm spatially on a map using the Bokeh plotting libaray.

    Args:
        asset_df(:obj:`pd.DataFrame`): PlantData.asset object containing the asset metadata.
        tile_name(:obj:`str`): tile set to be used for the underlay, e.g. OpenMap, ESRI, OpenTopoMap
        plot_width(:obj:`scalar`): width of plot
        plot_height(:obj:`scalar`): height of plot
        marker_size(:obj:`scalar`): size of markers
        kwargs_for_figure(:obj:`dict`): additional figure options for advanced users, see Bokeh docs
        kwargs_for_marker(:obj:`dict`): additional marker options for advanced users, see Bokeh docs. We have some custom behavior around the "fill_color" attribute. If "fill_color" is not defined, OpenOA will use an internally defined color pallete. If "fill_color" is the name of a column in the asset table, OpenOA will use the value of that column as the marker color. Otherwise, "fill_color" is passed through to Bokeh.

    Returns:
        Bokeh_plot(:obj:`axes handle`): windfarm map

    """

    # See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
    MAP_TILES = {
        "OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
        "ESRI": WMTSTileSource(
            url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"
        ),
        "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png"),
    }


    # Define default and then update figure and marker options based on kwargs
    figure_options = {
        "tools": "save,hover,pan,tap,wheel_zoom,reset,help",
        "x_axis_label": "Longitude",
        "y_axis_label": "Latitude",
        "match_aspect": True,
        "tooltips": [("nodeId", "@id"), 
                     ("plantName", "@name"), 
                     ("source", "@source"), 
                     ("capacity MW", "@generator_output_electricity"), 
                     ("COD", "@start_date"),
                     ("hub_height", "@height_hub"),
                     ("type", "@generator_type"),
                     ("rotor_diameter", "@rotor_diameter"),
                     ("manufacturer", "@manufacturer"),
                     ("model", "@model"),
                     ("(Lat,Lon)", "@coordinates")],
    }
    figure_options.update(kwargs_for_figure)

    marker_options = {
        "marker": "circle_y",
        "line_width": 1,
        "alpha": 0.8,
        "fill_color": "data_rank_colour",
        "line_color": "auto_line_color",
        "legend_group": "data_rank",
    }
    marker_options.update(kwargs_for_marker)

    # Create an appropriate fill color map and contrasting line color
    if marker_options["fill_color"] == "auto_fill_color":
        color_grouping = marker_options["legend_group"]

        df_points = df_points.sort_values(color_grouping)

        if len(set(df_points[color_grouping])) <= 10:
            color_palette = list(Category10[10])
        else:
            color_palette = viridis(len(set(df_points[color_grouping])))

        color_mapping = dict(zip(set(df_points[color_grouping]), color_palette))
        df_points["auto_fill_color"] = df_points[color_grouping].map(color_mapping)
        df_points["auto_fill_color"] = df_points["auto_fill_color"].apply(color_to_rgb)
        df_points["auto_line_color"] = [
            "black" if luminance(color) > 0.5 else "white" for color in df_points["auto_fill_color"]
        ]

    else:
        if marker_options["fill_color"] in df_points.columns:
            df_points[marker_options["fill_color"]] = df_points[marker_options["fill_color"]].apply(
                color_to_rgb
            )
            df_points["auto_line_color"] = [
                "black" if luminance(color) > 0.5 else "white"
                for color in df_points[marker_options["fill_color"]]
            ]

        else:
            df_points["auto_line_color"] = "black"

    # Create the bokeh data source
    source = ColumnDataSource(df_points)

    # Create a bokeh figure with tiles
    plot_map = figure(
        width=plot_width,
        height=plot_height,
        x_axis_type="mercator",
        y_axis_type="mercator",
        **figure_options,
    )

    plot_map.add_tile(MAP_TILES[tile_name])

    # Plot the asset devices
    renderer = plot_map.scatter(x="x", y="y", source=source, size=marker_size, **marker_options)

    renderer.selection_glyph = None
    renderer.nonselection_glyph = None

    url = "https://www.openstreetmap.org/node/@id/"
    taptool = plot_map.select(type=TapTool)
    taptool.callback = OpenURL(url=url)

    return plot_map

In [7]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [8]:

file_name = 'data/kvs'

if os.path.isfile(file_name):
    # open a file, where you stored the pickled data
    file = open(file_name, 'rb')

    # dump information to that file
    kvs = pickle.load(file)

    # close the file
    file.close()

else:
    kvs = osm.toolkit.get_osm_kvs()  
        
    # open a file, where you ant to store the data
    file = open(file_name, 'wb')

    # dump information to that file
    pickle.dump(kvs, file)

    # close the file
    file.close()


In [9]:
tags = Tags(kvs)

In [10]:
area_Guernsey = "(49.4096, -2.6875,49.5194, -2.4901)"
area_Ealing = "(51.487, -0.351, 51.5335, -0.2655)"
area_britishisles = "(48.327, -12.063, 61.428, 1.78)"
area_Glasgow = "(55.5683, -4.9507, 56.1318, -3.6639)"
area_Whitelee = "(55.5, -4.4364, 55.7591, -4.1)"
area_Wales = "(51.221, -5.559, 53.619, -2.845)"
area_Shetlands = "(59.8034, -2.4527, 60.9331, -0.4257)"
area_London = "(51.448, -0.2554, 51.6061, 0.0714)"
area_Nigeria = "(2,2,15,15)"
# area_Nigeria = "(9,4.6,10,4.8)" # used for testing
area_Nigeria = "(-40,-20,20,55)"
area_Whitelee = "(55.61,-4.46,55.76,-4.05)"

In [11]:
df_wind_turbines = osm.toolkit.get_osm_data(key=tags.generator__method,tag=tags.generator__method.wind_turbine,area=area_Whitelee,output="center")


In [12]:
df_wind_turbines.head(5)

,type,id,lat,lon,tags,generator:method,generator:source,operator,power,generator:output:electricity,generator:type,name,source,generator:model,height,operator:wikidata,manufacturer,model,key,tag
0,node,917143850,55.754008,-4.162368,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,Ecotricity,generator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine
1,node,1210840754,55.709544,-4.340662,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,2.3 MW,horizontal_axis,40,survey,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine
2,node,1210840811,55.691094,-4.232058,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,NaN,horizontal_axis,104,survey,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine
3,node,2029840477,55.648528,-4.325159,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,2.3 MW,horizontal_axis,D169,bing,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine
4,node,2029840482,55.648699,-4.315897,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,2.3 MW,horizontal_axis,C165,bing,NaN,NaN,NaN,NaN,NaN,generator:method,wind_turbine


In [13]:
df_wind_turbines["x"], df_wind_turbines["y"] = TRANSFORM_4326_TO_3857.transform(df_wind_turbines["lat"], df_wind_turbines["lon"])
df_wind_turbines["coordinates"] = tuple(zip(df_wind_turbines["lat"], df_wind_turbines["lon"]))

df_turbines_expanded = df_wind_turbines.rename(columns=lambda x: re.sub(':','_',x))

In [14]:
required_columns = set(["source","generator_output_electricity","start_date","height_hub","generator_type","rotor_diameter","manufacturer","model"])
missing_columns = list(required_columns - set(df_wind_turbines.columns))

In [15]:
df_turbines_expanded[missing_columns] = np.nan

In [16]:
df_turbines_expanded.head(5)

,type,id,lat,lon,tags,generator_method,generator_source,operator,power,generator_output_electricity,...,manufacturer,model,key,tag,x,y,coordinates,start_date,height_hub,rotor_diameter
0,node,917143850,55.754008,-4.162368,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,Ecotricity,generator,NaN,...,NaN,NaN,generator:method,wind_turbine,-463352.675122,7.509601e+06,"(55.7540076, -4.1623679)",NaN,NaN,NaN
1,node,1210840754,55.709544,-4.340662,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,NaN,...,NaN,NaN,generator:method,wind_turbine,-483200.305810,7.500810e+06,"(55.7095444, -4.3406622)",NaN,NaN,NaN
2,node,1210840811,55.691094,-4.232058,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,NaN,...,NaN,NaN,generator:method,wind_turbine,-471110.519304,7.497165e+06,"(55.6910937, -4.2320578)",NaN,NaN,NaN
3,node,2029840477,55.648528,-4.325159,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,NaN,...,NaN,NaN,generator:method,wind_turbine,-481474.497480,7.488763e+06,"(55.6485277, -4.325159)",NaN,NaN,NaN
4,node,2029840482,55.648699,-4.315897,"{'generator:method': 'wind_turbine', 'generato...",wind_turbine,wind,NaN,generator,NaN,...,NaN,NaN,generator:method,wind_turbine,-480443.478620,7.488797e+06,"(55.6486986, -4.3158972)",NaN,NaN,NaN


In [17]:
df_turbines_expanded["data_rank_colour"] = "sienna"
df_turbines_expanded["data_rank"] = "bronze"

In [18]:
gold_data_list = ["source","generator_output_electricity","start_date","height_hub","generator_type","rotor_diameter","manufacturer","model"]
silver_data_list = ["height_hub","rotor_diameter"]

In [19]:
df_turbines_expanded[silver_data_list].isnull().sum(axis=1)

df_turbines_expanded.loc[df_turbines_expanded[silver_data_list].isnull().sum(axis=1)==0,"data_rank_colour"] = "silver"
df_turbines_expanded.loc[df_turbines_expanded[silver_data_list].isnull().sum(axis=1)==0,"data_rank"] = "silver"

In [20]:
df_turbines_expanded[gold_data_list].isnull().sum(axis=1)

df_turbines_expanded.loc[df_turbines_expanded[gold_data_list].isnull().sum(axis=1)==0,"data_rank_colour"] = "gold"
df_turbines_expanded.loc[df_turbines_expanded[gold_data_list].isnull().sum(axis=1)==0,"data_rank"] = "gold"

In [21]:
df_turbines_expanded = df_turbines_expanded.drop(columns="tags")

In [25]:
df_turbines_expanded

,type,id,lat,lon,generator_method,generator_source,operator,power,generator_output_electricity,generator_type,...,tag,x,y,coordinates,start_date,height_hub,rotor_diameter,data_rank_colour,data_rank,auto_line_color
0,node,917143850,55.754008,-4.162368,wind_turbine,wind,Ecotricity,generator,NaN,NaN,...,wind_turbine,-463352.675122,7.509601e+06,"(55.7540076, -4.1623679)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
1,node,1210840754,55.709544,-4.340662,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-483200.305810,7.500810e+06,"(55.7095444, -4.3406622)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
2,node,1210840811,55.691094,-4.232058,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-471110.519304,7.497165e+06,"(55.6910937, -4.2320578)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
3,node,2029840477,55.648528,-4.325159,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-481474.497480,7.488763e+06,"(55.6485277, -4.325159)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
4,node,2029840482,55.648699,-4.315897,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-480443.478620,7.488797e+06,"(55.6486986, -4.3158972)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,node,8847868452,55.626942,-4.057746,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-451706.196225,7.484506e+06,"(55.6269417, -4.0577458)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
277,node,8847868454,55.624649,-4.053341,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-451215.900659,7.484054e+06,"(55.6246489, -4.0533414)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
278,node,8847868455,55.619490,-4.054581,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-451353.847772,7.483037e+06,"(55.6194896, -4.0545806)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white
279,node,8847868510,55.612038,-4.109144,wind_turbine,wind,NaN,generator,NaN,NaN,...,wind_turbine,-457427.839940,7.481568e+06,"(55.6120383, -4.1091442)",NaN,NaN,NaN,"(160, 82, 45)",bronze,white


In [22]:
map = plot_points(df_turbines_expanded)

show(map)


SerializationError: can't serialize <class '__main__.Tags'>

In [ ]:
map

figure(id='p1116', ...)